## MACHINE TRANSLATION

In [ ]:
import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import numpy as np
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install googletrans==3.1.0a0

     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 1.2 MB 9.7 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 65 kB 3.6 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=65d643e2d4c2184631af8bd1393ef2ba8a8927b8f20fefd4848ab24b68edcad1
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans


In [ ]:
# Imports
import pandas as pd

import googletrans
from googletrans import Translator


### Translate English articles to Swahili articles

In [ ]:
pd.set_option('max_colwidth', 300)
 
# how to get the supported language and their corresponing code
lang_df = pd.DataFrame.from_dict(googletrans.LANGUAGES,  orient='index', columns=['Language'])
print(len(googletrans.LANGUAGES))
lang_df.head(3)


107


,Language
af,afrikaans
sq,albanian
am,amharic


In [ ]:
# find the code for english and swahili
lang_df[lang_df.Language.isin(['english', 'swahili'])]

,Language
en,english
sw,swahili


In [ ]:
os.environ["gdrive_path"]='/content/drive/MyDrive/colab/MT_TRANSALATOR/'


In [ ]:
path='/content/drive/MyDrive/colab/MT_TRANSALATOR/'
en_sw= pd.read_csv(path+'/data.csv')


In [ ]:
en_sw=en_sw.sample(n=30000,random_state=42)
en_sw.shape

(30000, 2)

In [ ]:
en_df=en_sw.copy()

In [ ]:
en_df.shape

(30000, 2)

In [ ]:
en_df.rename(columns={'source_sentence':'EnglishText','target_sentence':'SwahiliText'},inplace=True)

In [ ]:
en_df=en_df[~pd.isnull(en_df['SwahiliText'])]
en_df=en_df[~pd.isnull(en_df['EnglishText'])]


In [ ]:
en_df.head()

,EnglishText,SwahiliText
244018,Both Ken and Barbara deeply appreciate having informed marriage mates whom they love and fully trust . *,Ken na Barbara wanathamini sana kuwa na wenzi wa ndoa ambao wanawapenda na kuwatumaini kabisa . *
552707,"( Jeremiah 31 : 34 ) In Israel , many just ignored Jehovah , saying in effect : “ Jehovah will not do good , and he will not do bad . ”","( Yeremia 31 : 34 ) Katika Israeli , wengi walimpuuza Yehova tu , wakisema hivi kumhusu : “ BWANA hatatenda mema , wala hatatenda mabaya . ”"
265432,I told my husband : “ Our daughter is not going to die .,Nilimwambia mume wangu : “ Binti yetu hatakufa .
596828,"Yet , all of us fail to meet God ’ s standards .",Lakini sisi sote hushindwa kufikia viwango vya Mungu .
673874,"What is more , as a result of Anna-Paula ’ s fine example and zealous preaching , her mother began studying the Bible and was soon baptized .","Zaidi ya hayo , kutokana na mfano bora na kuhubiri kwa bidii kwa Anna - Paula , mama yake alianza kujifunza Biblia naye akabatizwa muda mfupi baadaye ."


In [ ]:
en_df.shape

(29596, 2)

In [ ]:
en_df.head()

,EnglishText,SwahiliText
244018,Both Ken and Barbara deeply appreciate having informed marriage mates whom they love and fully trust . *,Ken na Barbara wanathamini sana kuwa na wenzi wa ndoa ambao wanawapenda na kuwatumaini kabisa . *
552707,"( Jeremiah 31 : 34 ) In Israel , many just ignored Jehovah , saying in effect : “ Jehovah will not do good , and he will not do bad . ”","( Yeremia 31 : 34 ) Katika Israeli , wengi walimpuuza Yehova tu , wakisema hivi kumhusu : “ BWANA hatatenda mema , wala hatatenda mabaya . ”"
265432,I told my husband : “ Our daughter is not going to die .,Nilimwambia mume wangu : “ Binti yetu hatakufa .
596828,"Yet , all of us fail to meet God ’ s standards .",Lakini sisi sote hushindwa kufikia viwango vya Mungu .
673874,"What is more , as a result of Anna-Paula ’ s fine example and zealous preaching , her mother began studying the Bible and was soon baptized .","Zaidi ya hayo , kutokana na mfano bora na kuhubiri kwa bidii kwa Anna - Paula , mama yake alianza kujifunza Biblia naye akabatizwa muda mfupi baadaye ."


In [ ]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
en_df['EnglishText']=en_df['EnglishText'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
en_df['SwahiliText']=en_df['SwahiliText'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [ ]:
en_df=en_df[~pd.isnull(en_df['EnglishText'])]
en_df.shape

(29596, 2)

In [ ]:
en_df.drop_duplicates(inplace=True)

In [ ]:
en_df.shape

(28168, 2)

In [ ]:
remove_digits = str.maketrans('', '', digits)

In [ ]:
# Remove quotes
en_df['EnglishText']=en_df['EnglishText'].apply(lambda x: re.sub("'", '', x))
en_df['SwahiliText']=en_df['SwahiliText'].apply(lambda x: re.sub("'", '', x))

In [ ]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
en_df['EnglishText']=en_df['EnglishText'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
en_df['SwahiliText']=en_df['SwahiliText'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [ ]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
en_df['EnglishText']=en_df['EnglishText'].apply(lambda x: x.translate(remove_digits))
en_df['SwahiliText']=en_df['SwahiliText'].apply(lambda x: x.translate(remove_digits))

# Remove extra spaces
en_df['EnglishText']=en_df['EnglishText'].apply(lambda x: x.strip())
en_df['SwahiliText']=en_df['SwahiliText'].apply(lambda x: x.strip())
en_df['EnglishText']=en_df['EnglishText'].apply(lambda x: re.sub(" +", " ", x))
en_df['SwahiliText']=en_df['SwahiliText'].apply(lambda x: re.sub(" +", " ", x))

In [ ]:
# Add start and end tokens to target sequences
en_df['SwahiliText']=en_df['SwahiliText'].apply(lambda x : 'START_ '+ x + ' _END')

In [ ]:
en_df.head()

,EnglishText,SwahiliText
244018,Both Ken and Barbara deeply appreciate having informed marriage mates whom they love and fully trust,START_ Ken na Barbara wanathamini sana kuwa na wenzi wa ndoa ambao wanawapenda na kuwatumaini kabisa _END
552707,Jeremiah In Israel many just ignored Jehovah saying in effect “ Jehovah will not do good and he will not do bad ”,START_ Yeremia Katika Israeli wengi walimpuuza Yehova tu wakisema hivi kumhusu “ BWANA hatatenda mema wala hatatenda mabaya ” _END
265432,I told my husband “ Our daughter is not going to die,START_ Nilimwambia mume wangu “ Binti yetu hatakufa _END
596828,Yet all of us fail to meet God ’ s standards,START_ Lakini sisi sote hushindwa kufikia viwango vya Mungu _END
673874,What is more as a result of AnnaPaula ’ s fine example and zealous preaching her mother began studying the Bible and was soon baptized,START_ Zaidi ya hayo kutokana na mfano bora na kuhubiri kwa bidii kwa Anna Paula mama yake alianza kujifunza Biblia naye akabatizwa muda mfupi baadaye _END


In [ ]:
### Get English and swahili Vocabulary
all_eng_words=set()
for eng in en_df.EnglishText:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_sw_words=set()
for sw in en_df.SwahiliText:
    for word in sw.split():
        if word not in all_sw_words:
            all_sw_words.add(word)

In [ ]:
len(all_eng_words)


27922

In [ ]:
all_sw_words


{'Baba',
 'Unamheshimuje',
 'mrija',
 'nilifahamu',
 'ulipoikaribisha',
 'amevutiwa',
 'itakaliwa',
 'kusanyiko',
 'Ebola',
 'zizi',
 'Waliokufa',
 'walipotoweka',
 'kukiondoaje',
 'asiingie',
 'Wedding',
 'hawadhihirishi',
 'tukijaza',
 'kufagiliwa',
 'inaishi',
 'MATOKEO',
 'USIKATE',
 'Hawakufanya',
 'ukamwomba',
 'Unifanye',
 'zimebeba',
 'pwekepweke',
 'wasimpe',
 'uliwekwa',
 'amesitawisha',
 'zenyewe',
 'uliozunguka',
 'yakakwama',
 'aliwakataza',
 'watavumilia',
 'ISA',
 'VEMA',
 'hatutawaona',
 'Sipa',
 'Greenewalt',
 'mamoja',
 'Asitawi',
 'nitairemba',
 'MIAMBA',
 'wanapenda',
 'Morse',
 'ajihusishapo',
 'litakufunulia',
 'ilipeleka',
 'akawakomboa',
 'Enenda',
 'Wakiri',
 'Postel',
 'KAZI',
 'zimefichwa',
 'aliyejulikana',
 'akitumikia',
 'Kedorlaoma',
 'tulioana',
 'kipato',
 'Nawaona',
 'ayarudie',
 'Walipendana',
 'Lefèvre',
 'Thamani',
 'Mbaya',
 'Bahati',
 'nilipoalikwa',
 'tuteseke',
 'lilirudi',
 'kunguru',
 'yahitajika',
 'vilivyokwezwa',
 'zimetokea',
 'Lima',
 'on

In [ ]:
en_df['lenght_EnglishText']=en_df['EnglishText'].apply(lambda x:len(x.split(" ")))
en_df['lenght_SwahiliText']=en_df['SwahiliText'].apply(lambda x:len(x.split(" ")))

In [ ]:
en_df[en_df['lenght_SwahiliText']>30].shape

(2300, 4)

In [ ]:
max_length_src=max(en_df.lenght_EnglishText)
max_length_tar=max(en_df.lenght_SwahiliText)
max_length_tar,max_length_src

(99, 109)

In [ ]:
en_df=en_df[en_df['lenght_EnglishText']<=20]
en_df=en_df[en_df['lenght_SwahiliText']<=20]

In [ ]:
max_length_src=max(en_df.lenght_EnglishText)
max_length_tar=max(en_df.lenght_SwahiliText)
max_length_tar,max_length_src

(20, 20)

In [ ]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_sw_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_sw_words)
num_encoder_tokens, num_decoder_tokens

(27922, 44523)

In [ ]:
num_decoder_tokens

44523

In [ ]:
num_decoder_tokens += 1 #for zero padding


In [ ]:
num_decoder_tokens

44524

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [ ]:
input_token_index


{'A': 1,
 'ABOUT': 2,
 'ABRAHAM': 3,
 'ABUSIVE': 4,
 'ACCEPT': 5,
 'ACCIDENT': 6,
 'ACCORDING': 7,
 'ACCOUNTS': 8,
 'ACCURACY': 9,
 'ACTIVITIES': 10,
 'ACTIVITY': 11,
 'ACTS': 12,
 'AD': 13,
 'ADAR': 14,
 'ADHD': 15,
 'ADJUST': 16,
 'ADOLESCENTS': 17,
 'ADVANCED': 18,
 'AFP': 19,
 'AFRICA': 20,
 'AFRICAN': 21,
 'AFTER': 22,
 'AFTERNOON': 23,
 'AGAINST': 24,
 'AGE': 25,
 'AGES': 26,
 'AIDS': 27,
 'AIDSrelated': 28,
 'AK': 29,
 'AL': 30,
 'ALBERT': 31,
 'ALCOHOL': 32,
 'ALEX': 33,
 'ALI': 34,
 'ALL': 35,
 'ALLOW': 36,
 'ALMOST': 37,
 'ALONG': 38,
 'ALS': 39,
 'ALSO': 40,
 'AM': 41,
 'AMERICAS': 42,
 'AN': 43,
 'ANCHOR': 44,
 'ANCIENT': 45,
 'AND': 46,
 'ANGMAGSSALIK': 47,
 'ANGÉLICA': 48,
 'ANIMALS': 49,
 'ANNE': 50,
 'ANNUAL': 51,
 'ANSWER': 52,
 'ANSWERED': 53,
 'ANSWERS': 54,
 'ANTHROPOLOGIST': 55,
 'ANTICHRIST': 56,
 'ANTWERP': 57,
 'ANY': 58,
 'AP': 59,
 'APPLICATION': 60,
 'APPLY': 61,
 'APPRECIATION': 62,
 'APPROACHES': 63,
 'APRIL': 64,
 'ARBOGA': 65,
 'ARDA': 66,
 'ARE': 67,
 'A

In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [ ]:
reverse_target_char_index

{1: 'A',
 2: 'AAMINI',
 3: 'ABRAHAMU',
 4: 'AD',
 5: 'ADARI',
 6: 'ADHABU',
 7: 'ADHD',
 8: 'ADN',
 9: 'AFP',
 10: 'AFRIKA',
 11: 'AFUNULIWA',
 12: 'AFYA',
 13: 'AGOSTI',
 14: 'AJ',
 15: 'AJILI',
 16: 'AK',
 17: 'AKATOKWA',
 18: 'AKILINI',
 19: 'AKIWA',
 20: 'AKSIDENTI',
 21: 'ALBERT',
 22: 'ALEX',
 23: 'ALI',
 24: 'ALICHEZA',
 25: 'ALIFANIKIWA',
 26: 'ALIFANYA',
 27: 'ALIJINYENYEKEZA',
 28: 'ALIKUWA',
 29: 'ALIONYESHAJE',
 30: 'ALIPOFIKA',
 31: 'ALIPONYA',
 32: 'ALIYEBATIZWA',
 33: 'ALIYELIJAZA',
 34: 'ALS',
 35: 'AMBAVYO',
 36: 'AMBAYO',
 37: 'AMERIKA',
 38: 'AMKARIBIAYE',
 39: 'AMKENI',
 40: 'AMRI',
 41: 'ANA',
 42: 'ANAJULIKANA',
 43: 'ANAKUJALI',
 44: 'ANATUJLI',
 45: 'ANAWEZA',
 46: 'ANDIKA',
 47: 'ANGMAGSSALIK',
 48: 'ANGÉLICA',
 49: 'ANNE',
 50: 'ANTWERP',
 51: 'AP',
 52: 'APINGWA',
 53: 'APRILI',
 54: 'ARBOGA',
 55: 'ARDA',
 56: 'ASHINDA',
 57: 'ASHURU',
 58: 'ATAWALA',
 59: 'ATHARI',
 60: 'ATL',
 61: 'AU',
 62: 'AUSTRALIA',
 63: 'AWALI',
 64: 'AWASAFISHA',
 65: 'AYUBU',
 66: 

In [ ]:
en_df = shuffle(en_df)
en_df.head(10)

,EnglishText,SwahiliText,lenght_EnglishText,lenght_SwahiliText
205357,In most places the program will begin each day with music at am,START_ Katika sehemu nyingi kila siku programu itaanza kwa muziki saa asubuhi _END,13,13
4210,Health and LifeStyle,START_ Afya na Mtindo Maisha _END,3,6
531563,Tommy began learning about the Bible from his parents when he was just five years old,START_ Tommy alianza kujifunza juu ya Biblia kutoka kwa wazazi wake alipokuwa na miaka mitano tu _END,16,17
601554,Also try to maintain a dignified but friendly atmosphere during the study,START_ Pia jaribu kudumisha hali yenye staha lakini ya kirafiki wakati wa funzo _END,12,14
330156,THE Bible says “ There exists a friend sticking closer than a brother ”,START_ BIBLIA inasema “ Kuna rafiki anayeshikamana na mtu kwa ukaribu zaidi kuliko ndugu ” _END,14,16
940792,That caused Hannah great anguish and distress,START_ Hilo lilimletea Hana uchungu mwingi na mfadhaiko _END,7,9
793919,What he learned changed his view of what is important in life,START_ Mambo aliyojifunza yalimsaidia kubadili maoni yake kuhusu kilicho muhimu maishani _END,12,12
62385,I organized a union for the train porters,START_ Nilipanga ushirika wa wafanyakazi wa garimoshi _END,8,8
868998,WRITE DOWN WHAT YOU LEARNED ABOUT,START_ ANDIKA MAMBO ULIYOJIFUNZA KUHUSU _END,6,6
518026,Only those who will make up the heavenly Kingdom are part of the seed of the woman,START_ Ni wale tu watakaofanyiza Ufalme wa kimbingu ndio sehemu ya mbegu ya mwanamke _END,17,15


#####Split the data into train and test¶


In [ ]:
X, y = en_df.EnglishText, en_df.SwahiliText
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((15002,), (3751,))

In [ ]:
y_train

717304                        START_ Alifikiria utamaduni na malezi yaliyoongoza njia ya kufikiri ya watu waliomsikiliza _END
169190                                                               START_ Kwa mfano hebu fikiria kunguru mwenye utundu _END
46043                                                                     START_ Utumizi Mbaya wa Kileo Katika Australia _END
638614                                                          START_ Hata hivyo imani yake iliathiri sana maisha yangu _END
333804                                                                                    START_ Blabu katika ukurasa wa _END
                                                                 ...                                                         
317436               START_ Lakini jambo la kusikitisha ni kwamba wahalifu wengi hawapewi adhabu yoyote na wenye mamlaka _END
736827                                                      START_ Wanapaswa kujifunza kuheshimu inavyofaa vichapo hiv

In [ ]:
encoder_input_data = np.zeros((2, max_length_src),dtype='float32')
decoder_input_data = np.zeros((2, max_length_tar),dtype='float32')
decoder_target_data = np.zeros((2, max_length_tar, num_decoder_tokens),dtype='float32')

In [ ]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)


In [ ]:
latent_dim = 300
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens+1, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
model.summary()
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 10

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 300)    8376900     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    13357500    input_2[0][0]                    
______________________________________________________________________________________________

In [ ]:
model.save('eng-to-swahili.h5')

In [ ]:
a, b = next(generate_batch())

In [ ]:
b

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [ ]:
val_samples

3751

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
117/117 [==============================] - 817s 7s/step - loss: 4.3320 - accuracy: 0.0993 - val_loss: 4.0336 - val_accuracy: 0.1129
Epoch 2/10
117/117 [==============================] - 803s 7s/step - loss: 3.7876 - accuracy: 0.1175 - val_loss: 3.9422 - val_accuracy: 0.1248
Epoch 3/10
117/117 [==============================] - 857s 7s/step - loss: 3.6186 - accuracy: 0.1300 - val_loss: 3.8492 - val_accuracy: 0.1352
Epoch 4/10
117/117 [==============================] - 785s 7s/step - loss: 3.4783 - accuracy: 0.1417 - val_loss: 3.8077 - val_accuracy: 0.1462
Epoch 5/10
117/117 [==============================] - 791s 7s/step - loss: 3.3597 - accuracy: 0.1551 - val_loss: 3.7807 - val_accuracy: 0.1561
Epoch 6/10
117/117 [==============================] - 790s 7s/step - loss: 3.2473 - accuracy: 0.1691 - val_loss: 3.7488 - val_accuracy: 0.1635
Epoch 7/10
117/117 [==============================] - 787s 7s/step - loss: 3.1376 - accuracy: 0.1833 - val_loss: 3.7234 - val_accuracy: 0.1731

In [ ]:
model.save_weights('nmt_weights.h5')


In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual swahili Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted swahili Translation:', decoded_sentence[:-4])

Input English sentence: Double Life I want to thank you for the article “ Young People Ask
Actual swahili Translation:  Maisha Maradufu Nataka kuwashukuru nyinyi kwa ile makala “ Vijana Huuliza 
Predicted swahili Translation:  “ Mimi ni “ Mimi ” — “ Mimi ” 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: Some years ago the cities of Sodom and Gomorrah were famous for depravity
Actual Hindi Translation:  Miaka ipatayo iliyopita majiji ya Sodoma na Gomora yalijulikana kwa upotovu wayo 
Predicted Hindi Translation:  Wakati wa miaka ya kwanza ya kwanza ya kwanza ya kw


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual swahili Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted swahili Translation:', decoded_sentence[:-4])

Input English sentence: Deliverance is at hand
Actual swahili Translation:  Kukombolewa kuko karibu 
Predicted swahili Translation:  Ni nini 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual swahili Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted swahili Translation:', decoded_sentence[:-4])

Input English sentence: I was encouraged to include three basic points in my witnessing
Actual swahili Translation:  Nilitiwa moyo nitie ndani mambo matatu ya msingi katika kutoa kwangu ushahidi 
Predicted swahili Translation:  Kisha ni lazima ikawa na watu wa kwanza na watu


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual swahili Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted swahili Translation:', decoded_sentence[:-4])


Input English sentence: Laboring in a carpentry shop
Actual swahili Translation:  Wafanya kazi ngumu katika karakana ya useremala 
Predicted swahili Translation:  Kisha ni hilo ni nini 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual swahili Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted swahili Translation:', decoded_sentence[:-4])

Input English sentence: Because they “ feared the true God ” — Exodus Genesis
Actual swahili Translation:  Kwa sababu “ walimwogopa Mungu wa kweli ” — Kutoka Mwanzo 
Predicted swahili Translation:  Kwa nini “ Mungu ” — Zaburi ya Mungu 


In [ ]:
k+=2
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+2].values[0])
print('Actual swahili Translation:', y_train[k:k+2].values[0][6:-4])
print('Predicted swahili Translation:', decoded_sentence[:-4])

Input English sentence: However it would be unloving and unreasonable for God to allow wrongdoing to continue forever
Actual swahili Translation:  Hata hivyo halingekuwa jambo la upendo na la akili ikiwa Mungu angeruhusu ubaya uendelee milele 
Predicted swahili Translation:  “ Kwa nini ni lazima kupata nguvu ya Mungu ” — Za


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual swahili Translation:', y_train[k:k+1].values[0][7:-4])
print('Predicted swahili Translation:', decoded_sentence[:-4])

Input English sentence: Such times together provide opportunity for interchanges that can be encouraging to both parent and child
Actual swahili Translation: Kutumia pindi kama hizo pamoja kwaandaa fursa za kuwasiliana zinazoweza kutia moyo mzazi na mtoto pia 
Predicted swahili Translation:  Ni nini kwa nini ni nini kwa nini 
